In [1]:
import requests
import requests.auth
import regex as re
from os.path  import basename
import random
import string
import json


client_auth = requests.auth.HTTPBasicAuth('AlFmsJweahucqYg4PwnQhQ', 'TiGAXDkqpxFKi9OYqTJiodHMGVt3Ew')
post_data = {"grant_type": "password", "username": "RedditorInCh1ef", "password": "Redd1tPassw0rd"}
headers = {"User-Agent": "ChangeMeClient/0.1 by RedditorInCh1ef"}
response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
auth_response = response.json()



In [55]:
def useable_link(i):
    """
    helper for get_links_from_post()
    """
    test = ['jpg','jpeg','png','imgur']
    for j in test:
        if j in i:
            return True
    return False

def get_links_from_post(postid):
    """
    Takes a post id and returns the possible img links
    """
    print(postid)
    comments = requests.get(f'http://oauth.reddit.com/r/photoshoprequest/comments/{postid}', headers=headers)
    c = comments.json()
    try:
        post_comments = [x['data']['body'] for x in c[1]['data']['children']] ##c[0] is post data it appears
        reg_link_find = 'https.+?(?=[)\n\]])'
        results = list(set([j for i in post_comments for j in re.findall(reg_link_find,i)]))
        results = [i for i in results if 'paypal' not in i] #no tipjar links
        results = [i for i in results if useable_link(i) ] #good links
        results = [i.replace('https://imgur.com/','https://i.imgur.com/').strip() for i in results]  #fix the imgur links these need file ext.
        results = [i + '.jpg' if ('imgur' in i) else i for i in results]
        return results
    except:
        return []

def get_random_string(length):
    # choose from all lowercase letter
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    return result_str

def download_image(img_url,length_of_file = 20):
    
    try:
        res=requests.get(img_url,allow_redirects = True,headers= {'Content-type': 'content_type_value'})
        response = requests.get(img_url)
        ext = response.headers['Content-Type'].split('/')[1]
        img_bytes= response.content # download bytes for a image
        rand_string = get_random_string(length_of_file)
        file_name = f"{rand_string}.{ext}"
        if ext in ('jpeg','png'):
            with open(file_name,"wb") as img_file:
                img_file.write(img_bytes)
            return file_name
        else:
            pass
    except Exception as e:
        pass

def save_and_document(pre_img_url,post_image_urls,caption):
    """
    Takes a candidate pre_img_url and the resultant list from get_links_from_post
    saves the pre img if there is at least one output img.  saves files (makes sure they are unique)
    and adds to the metadata file.
    """
    output_image_list = []
    #input_image = [] ## this will be a list, of the ones that work.
    
    for link_candidate in post_image_urls:
        file_name = download_image(link_candidate)
        if file_name is not None:
            output_image_list.append(file_name)
    
    if len(output_image_list) > 0:
        input_image_file = download_image(pre_img_url,length_of_file = 20)
    
        if input_image_file is not None:
            if len(input_image_file) > 0:
                for output_image in output_image_list:
                    dictionary = {"output_image": output_image, "input_image":input_image_file, "text":caption}
                    json_object = json.dumps(dictionary, indent=4)
                    with open("metadata.jsonl", "a") as outfile:
                        outfile.write(json_object)
            
    #return input_image_file

In [56]:
auth_response['access_token']

'38076669-a1Xmp9csvCfk3GERH50XGvSEfB9cJw'

In [57]:
token = auth_response['access_token']
auth = "bearer " + token
auth

'bearer 38076669-a1Xmp9csvCfk3GERH50XGvSEfB9cJw'

In [104]:
#t values hour, day, week, month, year, all
headers = {"Authorization": auth, "User-Agent": "I'm not a bot I'm just dumb"}
response = requests.get("https://oauth.reddit.com/r/photoshoprequest/top/?t=all?limit=10?before=t3_ud45sl", headers=headers)
x = response.json()

In [105]:
len(x['data']) ## this gets passed to the above request as an after

6

In [106]:
x

{'kind': 'Listing',
 'data': {'after': 't3_z54w6y',
  'dist': 25,
  'modhash': None,
  'geo_filter': '',
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'PhotoshopRequest',
     'selftext': '',
     'author_fullname': 't2_ntbreod',
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': "I lost my sweet Toki to cancer yesterday. I'd like a memorial photo of him please. Will tip.",
     'link_flair_richtext': [{'e': 'text', 't': 'Paid '},
      {'a': ':redditgold:',
       'e': 'emoji',
       'u': 'https://emoji.redditmedia.com/5knu5pox1guz_t5_3nqvj/redditgold'}],
     'subreddit_name_prefixed': 'r/PhotoshopRequest',
     'hidden': False,
     'pwls': 6,
     'link_flair_css_class': None,
     'downs': 0,
     'thumbnail_height': 140,
     'top_awarded_type': None,
     'hide_score': False,
     'name': 't3_z59rkr',
     'quarantine': False,
     'link_flair_text_color': 'light',
     'upvote_ratio

In [81]:
def posts_to_dataset(post_response):
    posts = post_response['data']['children']
    
    for post in posts:
        caption = post['data']['title']
        input_url = post['data']['url']
        postid = post['data']['id']
        print(post['data']['num_comments'])
        print(input_url)
        post_image_urls = get_links_from_post(postid)
        
        save_and_document(input_url,post_image_urls,caption)

In [107]:
x

{'kind': 'Listing',
 'data': {'after': 't3_z54w6y',
  'dist': 25,
  'modhash': None,
  'geo_filter': '',
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'PhotoshopRequest',
     'selftext': '',
     'author_fullname': 't2_ntbreod',
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': "I lost my sweet Toki to cancer yesterday. I'd like a memorial photo of him please. Will tip.",
     'link_flair_richtext': [{'e': 'text', 't': 'Paid '},
      {'a': ':redditgold:',
       'e': 'emoji',
       'u': 'https://emoji.redditmedia.com/5knu5pox1guz_t5_3nqvj/redditgold'}],
     'subreddit_name_prefixed': 'r/PhotoshopRequest',
     'hidden': False,
     'pwls': 6,
     'link_flair_css_class': None,
     'downs': 0,
     'thumbnail_height': 140,
     'top_awarded_type': None,
     'hide_score': False,
     'name': 't3_z59rkr',
     'quarantine': False,
     'link_flair_text_color': 'light',
     'upvote_ratio

In [108]:
posts_to_dataset(x)

48
https://i.redd.it/jcl3x7r9ab2a1.jpg
z59rkr
33
https://i.redd.it/lhgt05kztd2a1.jpg
z5epq5
31
https://i.redd.it/v42danvgnf2a1.jpg
z5n3y1
22
https://www.reddit.com/gallery/z5r92g
z5r92g
13
https://i.redd.it/akv6b108gg2a1.jpg
z5qfye
15
https://i.redd.it/hma072xpfd2a1.jpg
z5cu2h
3
https://www.reddit.com/gallery/z5gmgc
z5gmgc
7
https://i.redd.it/kshfdxy06c2a1.jpg
z577nf
4
https://i.redd.it/obvqetygrg2a1.jpg
z5rp1l
12
https://i.redd.it/irpeudeuce2a1.jpg
z5nyoh
8
https://www.reddit.com/gallery/z5fwpy
z5fwpy
11
https://www.reddit.com/gallery/z5b3cs
z5b3cs
8
https://i.redd.it/gpuygt99fc2a1.jpg
z588qm
4
https://i.redd.it/p9uk4zuc9f2a1.png
z5rogv
4
https://i.redd.it/3jf1elgdgg2a1.jpg
z5qgiv
19
https://i.redd.it/oe6j799y4g2a1.jpg
z5p5ai
7
https://i.redd.it/0xy10auz3g2a1.jpg
z5p1da
1
https://www.reddit.com/gallery/z5nel5
z5nel5
22
https://www.reddit.com/gallery/z5imix
z5imix
4
https://imgur.com/a/rtmtspm
z5ga3z
4
https://files.fm/u/k6prv5gn2
z5epth
20
https://imgur.com/a/7z23QPW
z5enqt
7
https://

In [308]:
x['data']['children'][1]['data']['title']  ##this is the text

TypeError: list indices must be integers or slices, not str

In [239]:
x['data']['children'][1]['data']['url']  ##i think this is the request (input) photo
##look at x['data']['children'][1]['data']['preview'], it has some stuff
##x['data']['children'][1]['data']['url_overridden_by_dest'] This one is identical in this case, maybe check both?

'https://i.redd.it/jcl3x7r9ab2a1.jpg'

In [37]:
x['data']['children'][1]['data']['num_comments'] ##check this before getting the comments

36

In [240]:
postid = x['data']['children'][1]['data']['id']

In [241]:
postid

'z59rkr'

In [242]:
#x = get_links_from_post(postid)

comments = requests.get(f'http://oauth.reddit.com/r/photoshoprequest/comments/{postid}', headers=headers)
c = comments.json()


In [244]:
x = get_links_from_post(postid)

In [250]:
x[4]

'https://i.imgur.com/XU6L4R9.jpg'

In [309]:
save_and_document('https://i.redd.it/jcl3x7r9ab2a1.jpg',x,"the caption for the picture")

'mliiidjjcbablupcugws.jpeg'

In [288]:
download_image('cdfg')

In [280]:
xx.headers['Content-Type'].split('/')[1]

'jpeg'

In [ ]:
##https://github.com/zuruoke/watermark-removal

In [ ]:
#https://i.imgur.com/XU6L4R9.jpeg

In [148]:
images = soup.findAll('meta')

In [206]:
output_image = "whatever"
input_image = "whatever_alse"
caption = "something else"
metadata_string = f'{{"output_image": {output_image}, "input_image":{input_image}, "text":{caption}}}'

In [207]:
metadata_string

'{"output_image": whatever, "input_image":whatever_alse, "text":something else}'